<a href="https://colab.research.google.com/github/tjturnage/radar/blob/main/Plot_NEXRAD_mosaics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Plot NEXRAD Reflectivity Mosaics**<br />

This allows the user to plot nexrad mosaic reflectivity images described at:
https://mesonet.agron.iastate.edu/docs/nexrad_mosaic/
<br />
<br />
The user can define the following:
- lat/lon box containing the plot
- one of two available reflectivity colormaps (additional maps available on request)
- which cities to plot (if any) and where to place the labels
- whether to plot major roads
- whether to plot county outlines
<br/> <br/>

&nbsp;&nbsp;*T.J. Turnage (NWS Grand Rapids, MI - thomas.turnage@noaa.gov)*
<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*24-Nov-2023 -- v0.99 Initial release*<br/>


In [7]:
# @title Install the needed GIS software
!pip install rasterio --quiet
!pip install cartopy --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 24.5 MB/s eta 0:00:00


In [40]:
# @title Configure your settings here
"""
Oct 31 2023:
first_plot_time = "20231031_1600"
last_plot_time = "20231031_1915"

Nov 06 2023:
first_plot_time = "20231106_1245"
last_plot_time = "20231106_1255"
lat_min = 42.0
lat_max = 44.55
lon_min = -87.50
lon_max = -84.55
fig_height = 7
fig_width = 5
plot_aspect_ratio = 1.25
"""


#############################################################
# Configuration Section
#############################################################

# This is in UTC time, must be 5 minute increments
# Example: 20231031_1630 = 31 October 2023 16:30 UTC
first_plot_time = "20230824_2245"
last_plot_time = "20230825_0245"

# Define your local UTC time shift and observed time zone
utc_minus_local_time = 4
time_zone = "EDT"

# plot lat/lon ranges
lat_min = 42.0
lat_max = 44.25
lon_min = -86.90
lon_max = -83.95

# reflectivity colormap selection
plot_cmap = "wdtd_z"  # options are "wdtd_z" and "winter_z"

# This will require experimentation on your part because the combination of
# your chosen lat/lon ranges and figure dimensions often result in some
# undesired map stretching. plot_aspect_ratio can also help with adjustments.
# plot_aspect_ratio > 1 shrinks horizontal scale and vice versa.

fig_height = 7
fig_width = 5
plot_aspect_ratio = 1.25

# You'll need to create your own cities dictionary and play around
# with the best offset position for each of the labels. For example,
# - positive "lat_offset" shifts the label north by that many degrees
# - negative "lon_offest" shifts the label east by that many degrees

plot_counties = True
plot_roads = True
plot_cities = True

cities = {
'Grand Rapids':{'lat':42.963,'lon':-85.668, 'lat_offset':0.04, 'lon_offset':0.06},
'Muskegon':{'lat':43.234,'lon':-86.248, 'lat_offset':0.04, 'lon_offset':0.06},
'Holland':{'lat':42.7875,'lon':-86.1089, 'lat_offset':-0.10, 'lon_offset':0.06},
'South Haven':{'lat':42.4031,'lon':-86.2736, 'lat_offset':0.04, 'lon_offset':0.06},
'Lansing':{'lat':42.7325,'lon':-84.5556, 'lat_offset':0.00, 'lon_offset':0.06},
'Kalamazoo':{'lat':42.292,'lon':-85.587, 'lat_offset':-0.10, 'lon_offset':0.06},
'Ludington':{'lat':43.955,'lon':-86.4525, 'lat_offset':0.04, 'lon_offset':0.06},
'Big Rapids':{'lat':43.698,'lon':-85.4836, 'lat_offset':0.04, 'lon_offset':0.06},
'Mount Pleasant':{'lat':43.5978,'lon':-84.7675, 'lat_offset':0.06, 'lon_offset':0.06},
           }

#############################################################
# End Configuration Section
#############################################################

In [32]:
# @title Ensure colormaps and GIS files are ready
import sys
import os
from os.path import exists
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import urllib.request
import requests
from bs4 import BeautifulSoup
import zipfile
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.io.shapereader as shpreader
import xarray as xr
from datetime import datetime
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")


def create_and_register_cmap(cmap_name, original_colors, position, rgb_input=True):
    """
    1) Converts list of color values from 8-bit (0-255 scale)
       to 0-1 decimal scale.

    2) The created final_color_list is combine with position list
       to create a matplotlib colormap.

    3) Registers the new colormap using the supplied cmap_name.

    Input
    ----------
             cmap_name : string
                         Name of the cmap you want to use and register

      original_ colors : list
                         8-bit tuples containing RGB values
                         example: [(127,256,192), (125,125,125)]

              position : list
                         node locations for color break points on
                         a 0 to 1 scale

            rgb_input : boolean
                        True: Color conversion from RGB to performed.
                       False: Color conversion from RGB is skipped.

    Returns
    -------
    None

    """
    if rgb_input:
        final_color_list = []
        for color in original_colors:
            converted_color_group = []
            for original_channel_value in color:
                new_channel_value = float(original_channel_value)/255
                converted_color_group.append(float(format(new_channel_value, '.3f')))
            converted_color_tuple = tuple(converted_color_group)
            final_color_list.append(converted_color_tuple)
    else:
        final_color_list = original_colors

    this_cmap = LinearSegmentedColormap.from_list(cmap_name, list(zip(position, final_color_list)))
    mpl.colormaps.register(cmap=this_cmap, force=True)
    return

###############################################################################
colors=[(0,0,0),(130,130,130),(95,189,207),(57,201,105),(57,201,105),(0,40,0),
        (9,94,9),(255,207,0),(255,207,0),(255,133,0),(255,0,0),(89,0,0),
        (255,245,255),(225,11,227),(164,0,247),(99,0,214),(5,221,224),
        (58,103,181),(255,255,255)]
position=[0.0,0.407,0.409,0.452,0.454,0.587,0.590,0.632,0.636,0.722,0.727,0.812,
          0.818,0.902,0.909,0.947,0.954,0.992,1.0]

create_and_register_cmap("wdtd_z", colors, position, rgb_input=True)
###############################################################################
colors=[(0,0,0),(70,70,70),(210,210,210),(0,120,255),(0,0,60),(255,255,0),
        (220,100,0),(220,20,0),(220,20,0)]
position = [0,32/122,50/122,54/122,60/122,64/122,72/122,82/122,1]

create_and_register_cmap("winter_z", colors, position)
###############################################################################

counties_url = "https://www2.census.gov/geo/tiger/TIGER2019/COUNTY/tl_2019_us_county.zip"
counties_url = "https://www.weather.gov/source/gis/Shapefiles/County/c_08mr23.zip"

url = "https://www.weather.gov/gis/Counties"
response = requests.get(url)

# Find name of latest available NWS Counties shapefile
soup = BeautifulSoup(response.content, "html.parser")
zip_links = soup.find_all("a", href=lambda href: href.endswith(".zip"))
if zip_links:
    zip_url = zip_links[-1]["href"]
    counties_url = os.path.join('https://www.weather.gov',str(zip_url)[1:])
    zip_file_name = counties_url.split("/")[-1]
    zip_file_root = zip_file_name[:-4]
    shapefile_name = f'{zip_file_root}.shp'
else:
    print("No zip files found on the website.")

if not exists(zip_file_name):
  urllib.request.urlretrieve(counties_url, "counties.zip")
  with zipfile.ZipFile('counties.zip', 'r') as zip_ref:
      zip_ref.extractall('.')


try:
  COUNTY_SHAPEFILE
except NameError:
  #reader = shpreader.Reader("tl_2019_us_county.shp")
  reader = shpreader.Reader(shapefile_name)
  features = list(reader.geometries())
  COUNTY_SHAPEFILE = cfeature.ShapelyFeature(features, ccrs.PlateCarree())

# download and read road shapefile only if plot_roads=True
if plot_roads:
  road_shapefile_name = "tl_2019_us_primaryroads.shp"
  road_url = "https://www2.census.gov/geo/tiger/TIGER2019/PRIMARYROADS/tl_2019_us_primaryroads.zip"
  if not exists(road_shapefile_name):
    urllib.request.urlretrieve(road_url, "roads.zip")
    with zipfile.ZipFile('roads.zip', 'r') as zip_ref:
      zip_ref.extractall('.')
  try:
    ROAD_SHAPEFILE
  except NameError:
    #reader = shpreader.Reader("tl_2019_us_primary_roads.shp")
    road_reader = shpreader.Reader(road_shapefile_name)
    features = list(road_reader.geometries())
    ROAD_SHAPEFILE = cfeature.ShapelyFeature(features, ccrs.PlateCarree())

# for these pngs, we already know their values of lat/lon, ranges, dx, dy
x = np.arange(-126.0, -65.0, 0.005)
y = np.arange(23.0, 50.0, 0.005)
coords = {'longitude': x, 'latitude': y}

plt.rcParams['font.size'] = 10
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['lines.markersize'] = 6
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['text.color'] = 'white'
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['scatter.marker'] = 'o'


In [ ]:
# @title Create your images!
def create_valid_times(initial_time,final_time):
  if int(initial_time[-2:])%5 != 0:
    print("Initial time must be to nearest 5 minutes!")
  start_time = datetime.strptime(initial_time, "%Y%m%d_%H%M")
  end_time = datetime.strptime(final_time, "%Y%m%d_%H%M")

  valid_times = []
  i = 0
  this_time = start_time + timedelta(minutes=5)*i
  valid_times.append(this_time)
  while this_time < end_time:
    i += 1
    this_time = start_time + timedelta(minutes=5)*i
    valid_times.append(this_time)

  return valid_times

valid_times = create_valid_times(first_plot_time,last_plot_time)

# ------------- Download pngs, create plots, save images -------------

PNG_URL_BASE = "https://mesonet.agron.iastate.edu/archive/data"

# create fresh images directory to store new images to
os.system('rm -rf images')
os.mkdir('images')

for plot_time in valid_times:
  local_time = plot_time - timedelta(hours=utc_minus_local_time)

  tail = datetime.strftime(plot_time,"%Y/%m/%d/GIS/uscomp/n0q_%Y%m%d%H%M.png")
  downloaded_file_name = tail[-16:]
  if not exists(downloaded_file_name):
    url = os.path.join(PNG_URL_BASE,tail)
    map = urllib.request.urlretrieve(url,downloaded_file_name)

  with rasterio.open(downloaded_file_name) as src:
    # Get the image data and metadata
    image_data = src.read()

    #slice (x,y) cartesian array out of current (1,x,y) array
    image_new = image_data[0,:,:]

    # convert 8-bit (0-255) values to reflectivity values as described at
    # https://mesonet.agron.iastate.edu/GIS/rasters.php?rid=2
    image_new = (image_new * 0.5) - 32.5

    #flip data vertically to move origin from upper left to lower left
    image_flipped = np.flipud(image_new)

    # Add data to xarray with the defined coordinate system
    data = xr.DataArray(image_flipped, coords=coords, dims=['latitude', 'longitude'])
    data.attrs["units"] = "dBZ"
    data.attrs["long_name"] = "Reflectivity"

    # Determine region you want to plot
    subset = data.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max))

    fig, ax = plt.subplots(1,1,figsize=(fig_height,fig_width),subplot_kw={'projection': ccrs.PlateCarree()})
    subset.plot(ax=ax, cmap=plot_cmap, vmin=-32, vmax=95)
    ax.set_aspect(plot_aspect_ratio)
    if plot_counties:
      ax.add_feature(COUNTY_SHAPEFILE, facecolor='none', edgecolor='#cccccc', linewidth=0.7)

    if plot_roads:
      ax.add_feature(ROAD_SHAPEFILE, facecolor='none', edgecolor='cyan', linewidth=0.5)
    ax.add_feature(cfeature.STATES, facecolor='none', edgecolor='white', linewidth=1.3)
    ax.set_title(datetime.strftime(local_time,f'%b %d, %Y -- %I:%M %p {time_zone}'), color='black')
    plt.tight_layout()
    target_fname = f'images/cropped_{downloaded_file_name}'
    if plot_cities:
      for key in cities:
        c = cities[key]
        lat,lon = c['lat'], c['lon']
        if ((lat > lat_min) & (lat < lat_max) & (lon > lon_min) & (lon < lon_max)):
          plt.plot([lon], [lat], color='#ff9999', linewidth=1, marker='o', markersize=6, zorder=10)
          plt.plot([lon], [lat], color='white', linewidth=1, marker='o', markersize=4, zorder=10)
          plt.text(lon + c['lon_offset'], lat + c['lat_offset'], key, color='white', zorder=11)
    plt.savefig(target_fname,bbox_inches='tight')

# tar images files together so you can download single file
os.system('tar cvf images.tar images/')